In [6]:
# Define the function f(a) = y^a mod N for y = 2 and N = 39
def modular_exponentiation(y, N, limit=20):
    results = {}
    for a in range(limit):
        results[a] = (y ** a) % N
    return results

# Verify the function f(a) for y = 2, N = 39
y = 4
N = 39
limit = 20  # Calculate up to 20 values of a
modular_exponentiation(y, N, limit)


{0: 1,
 1: 2,
 2: 4,
 3: 8,
 4: 16,
 5: 32,
 6: 25,
 7: 11,
 8: 22,
 9: 5,
 10: 10,
 11: 20,
 12: 1,
 13: 2,
 14: 4,
 15: 8,
 16: 16,
 17: 32,
 18: 25,
 19: 11}

In [65]:
# Imports of the Qiskit basic functionalities
%matplotlib inline
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from math import gcd
from fractions import Fraction
import numpy as np

def qft_dagger_rz(n):
    """Quantum Fourier Transform dagger using Rz gates."""
    qc = QuantumCircuit(n)
    
    # Reverse the order of the qubits
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    
    # Apply the QFT dagger gates
    for j in range(n):
        for m in range(j):
            angle = -np.pi / (2 ** (j - m))
            qc.rz(angle, j)  # Apply Rz gate for phase shift
            qc.cx(m, j)     # Add a CNOT gate to complete the phase interaction
            qc.rz(-angle, j)  # Inverse phase shift after CNOT
            qc.cx(m, j)     # Add the CNOT back to complete the controlled phase
        qc.h(j)
    
    return qc

def c_amod39(a, power):
    """Controlled modular exponentiation by repeated composition."""
    U = QuantumCircuit(6)
    for iteration in range(power):
        U = U.compose(amod39(a))
    U = U.to_gate()
    U.name = "%i^%i mod 39" % (a, power)
    c_U = U.control()
    return c_U

def amod39(a):
    """Generic modular multiplication circuit."""
    U = QuantumCircuit(6)
    for i in range(6):
        if (a ** (2 ** i)) % 39 != 0:
            U.x(i)
    return U

def qpe_amod39(a):
    """Quantum Phase Estimation for modular exponentiation."""
    n_count = 7
    qc = QuantumCircuit(n_count + 6, n_count)
    for q in range(n_count):
        qc.h(q)
    qc.x(5 + n_count)
    for q in range(n_count):
        qc.append(c_amod39(a, 2 ** q), [q] + [i + n_count for i in range(6)])
    qc.append(qft_dagger_rz(n_count), range(n_count))
    qc.measure(range(n_count), range(n_count))
    return qc

def simulate_shors(N):
    """Main simulation function for Shor's algorithm."""
    simulator = BasicSimulator()

    a = 5
    qc = qpe_amod39(a)
    
    print("Quantum Circuit (Text):")
    print(qc.draw(output='text'))

    # Simulate the circuit
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()
    
    # Post-processing
    measured_phases = []
    for output in counts:
        decimal = int(output, 2)
        phase = decimal / (2 ** 8)
        measured_phases.append(phase)

    fractions = [Fraction(phase).limit_denominator(N) for phase in measured_phases]
    rs = [frac.denominator for frac in fractions]

    # Remove duplicates and sort
    rs = list(set(rs))
    rs.sort()
    
    # Attempt to find the factors
    for r in rs:
        if r % 2 == 0:
            guess1 = gcd(a ** (r // 2) - 1, N)
            guess2 = gcd(a ** (r // 2) + 1, N)
            if guess1 != 1 and guess1 != N:
                print(f"Factors of {N} found: {guess1} and {N // guess1}")
                return
            if guess2 != 1 and guess2 != N:
                print(f"Factors of {N} found: {guess2} and {N // guess2}")
                return

    print("Failed to find factors.")

N = 39
print(f"Factors of {N}:")
simulate_shors(N)


Factors of 39:
Quantum Circuit (Text):
      ┌───┐                                                            »
 q_0: ┤ H ├───────■────────────────────────────────────────────────────»
      ├───┤       │                                                    »
 q_1: ┤ H ├───────┼──────────────■─────────────────────────────────────»
      ├───┤       │              │                                     »
 q_2: ┤ H ├───────┼──────────────┼──────────────■──────────────────────»
      ├───┤       │              │              │                      »
 q_3: ┤ H ├───────┼──────────────┼──────────────┼──────────────■───────»
      ├───┤       │              │              │              │       »
 q_4: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼───────»
      ├───┤       │              │              │              │       »
 q_5: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼───────»
      ├───┤       │              │              │              │       »
 q_6: ┤ H ├─

In [50]:
qc = qft_dagger_rz(8)
print("Quantum Circuit (Text):")
print(qc.draw(output='text'))

Quantum Circuit (Text):
            ┌───┐                                                           »
q_0: ─X─────┤ H ├────────────────────────■─────────────────────■─────────■──»
      │     └───┘      ┌──────────┐    ┌─┴─┐     ┌─────────┐ ┌─┴─┐┌───┐  │  »
q_1: ─┼───────X────────┤ Rz(-π/2) ├────┤ X ├─────┤ Rz(π/2) ├─┤ X ├┤ H ├──┼──»
      │       │        └──────────┘ ┌──┴───┴───┐ └─────────┘ └───┘└───┘┌─┴─┐»
q_2: ─┼───────┼─────────────X───────┤ Rz(-π/4) ├───────────────────────┤ X ├»
      │       │             │       └──────────┘ ┌──────────┐          └───┘»
q_3: ─┼───────┼─────────────┼────────────X───────┤ Rz(-π/8) ├───────────────»
      │       │             │            │      ┌┴──────────┤               »
q_4: ─┼───────┼─────────────┼────────────X──────┤ Rz(-π/16) ├───────────────»
      │       │             │      ┌───────────┐└───────────┘               »
q_5: ─┼───────┼─────────────X──────┤ Rz(-π/32) ├────────────────────────────»
      │       │       ┌───────────┐└────

In [48]:
qc = amod39(5)
print("Quantum Circuit (Text):")
print(qc.draw(output='text'))

Quantum Circuit (Text):
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     ├───┤
q_3: ┤ X ├
     ├───┤
q_4: ┤ X ├
     ├───┤
q_5: ┤ X ├
     └───┘


In [60]:
import numpy as np
from scipy.linalg import expm

# Define Pauli matrices
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
identity = np.eye(2, dtype=complex)

# Tensor product function
def tensor_product(matrices):
    result = matrices[0]
    for mat in matrices[1:]:
        result = np.kron(result, mat)
    return result

# Define H_x, H_y, H_z using tensor products
Hx = (
    tensor_product([sigma_x, sigma_x, identity]) +
    tensor_product([sigma_x, identity, sigma_x]) +
    tensor_product([identity, sigma_x, sigma_x])
)

Hy = (
    tensor_product([sigma_y, sigma_y, identity]) +
    tensor_product([sigma_y, identity, sigma_y]) +
    tensor_product([identity, sigma_y, sigma_y])
)

Hz = (
    tensor_product([sigma_z, sigma_z, identity]) +
    tensor_product([sigma_z, identity, sigma_z]) +
    tensor_product([identity, sigma_z, sigma_z])
)

# Full Hamiltonian
H = Hx + Hy + Hz

# Define time and initial state
J = 1  # Coupling constant
t = np.pi / (2 * J)  # Time
psi0 = np.array([0, 0, 0, 1, 0, 0, 0, 0], dtype=complex)  # |001> state

# Exact evolution U1
U1 = expm(-1j * J * t * H)
psi1 = np.dot(U1, psi0)

# Sequential decomposition U2
U2 = np.dot(expm(-1j * J * t * Hx), np.dot(expm(-1j * J * t * Hy), expm(-1j * J * t * Hz)))
psi2 = np.dot(U2, psi0)

# Trotter-Suzuki approximation U3
n_steps = 50
delta_t = t / n_steps
U3_step = np.dot(
    expm(-1j * J * delta_t * Hx),
    np.dot(expm(-1j * J * delta_t * Hy), expm(-1j * J * delta_t * Hz))
)
U3 = np.linalg.matrix_power(U3_step, n_steps)
psi3 = np.dot(U3, psi0)

# Display results
print("psi1 (Exact):", psi1)
print("psi2 (Sequential):", psi2)
print("psi3 (Trotter-Suzuki):", psi3)


psi1 (Exact): [ 0.00000000e+00+0.j          0.00000000e+00+0.j
  0.00000000e+00+0.j          1.35798641e-17-0.33333333j
  0.00000000e+00+0.j         -3.59124515e-16+0.66666667j
  2.34279725e-16+0.66666667j  0.00000000e+00+0.j        ]
psi2 (Sequential): [-8.55879796e-33+5.48417043e-33j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  1.27983421e-16-1.00000000e+00j
  0.00000000e+00+0.00000000e+00j -9.98367309e-17-2.26823074e-17j
 -1.89215761e-16-9.98367309e-17j  0.00000000e+00+0.00000000e+00j]
psi3 (Trotter-Suzuki): [ 0.00615898-0.09789414j  0.        +0.j          0.        +0.j
 -0.00205299-0.33818557j  0.        +0.j         -0.00205299+0.66181443j
 -0.00205299+0.66181443j  0.        +0.j        ]


In [61]:
# Define the exact state for comparison
psi_exact = np.array([0, 0, 0, -1j/3, 0, 2j/3, 2j/3, 0], dtype=complex)

# Function to calculate overlap
def calculate_overlap(psi_exact, psi):
    return np.abs(np.dot(np.conjugate(psi_exact), psi))**2

# Calculate overlaps
overlap1 = calculate_overlap(psi_exact, psi1)
overlap2 = calculate_overlap(psi_exact, psi2)
overlap3 = calculate_overlap(psi_exact, psi3)

# Display results
print("Overlap with exact state:")
print(f"Overlap1 (Exact Evolution): {overlap1}")
print(f"Overlap2 (Sequential Decomposition): {overlap2}")
print(f"Overlap3 (Trotter-Suzuki): {overlap3}")

Overlap with exact state:
Overlap1 (Exact Evolution): 0.9999999999999998
Overlap2 (Sequential Decomposition): 0.11111111111111106
Overlap3 (Trotter-Suzuki): 0.9903232867273773


In [63]:
8 / 2 * (2 * 2)

16.0